In [1]:
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt


In [2]:
def function(x, y, z):
    '''Define the polynomial function f(x, y, z) here.'''
    return  x**2 + y**2 + z**2 -1  

In [3]:
def gradient(f):
    '''Calculates the symbolic gradient of the function f.'''
    x, y, z = sp.symbols('x y z')
    f_sym = f(x, y, z)
    df_dx = sp.diff(f_sym, x)
    df_dy = sp.diff(f_sym, y)
    df_dz = sp.diff(f_sym, z)
    return sp.lambdify((x, y, z), df_dx), sp.lambdify((x, y, z), df_dy), sp.lambdify((x, y, z), df_dz)

In [4]:
def grid(x0,y0,z0,B, Num_base_points):
    '''returns three 3-Dimensional arrays representing the X and Y coordinates of all the points in the regular
    cubic grid with centre (x0,y0,z0), side 2B and Num_base_points^3 points.
    '''
    x = np.linspace(x0 - B, x0 + B, Num_base_points)
    y = np.linspace(y0 - B, y0 + B, Num_base_points) 
    z = np.linspace(y0 - B, y0 + B, Num_base_points) 
    
    return np.meshgrid(x, y, z, indexing='xy')

In [5]:
def norm_f_on_grid(grid):
    """returns a 3-Dimensional array with the absolute value of the function on the points in the 
    (Num_base_points) x (Num_base_points) x (Num_base_points) mesh."""
    return np.absolute(  function(grid[0],grid[1],grid[2]) )

In [6]:
def norm_grad_on_grid(grid):
    """returns a 3-Dimensional array with the value of the L_1 norm of gradient of the function on the points in the 
    (Num_base_points) x (Num_base_points) mesh."""
    return np.absolute(gradient(function)[0](grid[0],grid[1],grid[2]) ) + np.absolute(gradient(function)[1](grid[0],grid[1],grid[2]) ) + np.absolute(gradient(function)[2](grid[0],grid[1],grid[2]) ) 

In our algorithm, we are interested in the points on the grid, where the norm of f is less than $\sqrt N* \epsilon /2 *M$ and the norm of the grad is less than $N^{3/2}*\epsilon*K$. We would like to see how small the size of the grid gets in order to bound the gradient of f on the zero set of f. Here $M$ is un upper bound for the $L_1$ norm of the gradient of $f$ on the bounding box and $K$ is the largest eigenvalue of the hessian of $f$ on the bounding box. We can calculate this parameters numerically.

In [7]:
def hessian(f):
    '''Calculate the symbolic Hessian matrix of the function f.'''
    x, y, z = sp.symbols('x y z')
    f_sym = f(x, y, z)
    hessian_matrix = sp.hessian(f_sym, (x, y, z))
    return sp.lambdify((x, y, z), hessian_matrix)

def largest_eigenvalue(hessian_func, X, Y, Z):
    '''Compute the largest eigenvalue of the Hessian at each point on the grid.'''
    largest_eigenvalues = np.zeros_like(X)
    
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            for k in range(X.shape[2]):
                hessian_matrix = np.array(hessian_func(X[i, j, k], Y[i, j, k], Z[i, j, k]), dtype=np.float64)
                eigenvalues = np.linalg.eigvals(hessian_matrix)
                largest_eigenvalues[i, j, k] = np.max(eigenvalues)
    
    return largest_eigenvalues


In [8]:
L = 20 # Number of points along each axis
B = 2
init_point = np.array([0,0,0])


X, Y, Z = grid(init_point[0], init_point[1], init_point[2], B, L)


# maximum of all the largest eigenvalues
K = np.max(largest_eigenvalue(hessian(function), X, Y, Z))


print("Maximum of the largest eigenvalues:\n", K)

Maximum of the largest eigenvalues:
 2.0


In [9]:
M = np.max(norm_grad_on_grid([X, Y,Z]))
print("Upper bound of norm of grad_f:\n", M)

Upper bound of norm of grad_f:
 12.0


In [10]:
#M=12 abs bound on the gradient of f
#K=2 abs bound on the hessian of f

def dubious_points(mesh,a_tolerance):
    '''given a grid `mesh' and an absolute tolerance (our epsilon), it returns a list with the coordinates
    of the points on which the abs value of the function and its gradient are smaller than a constant times a_tolerance''' 
    threshold_f = np.sqrt(3)*a_tolerance/2*M
    threshold_grad = 3*np.sqrt(3)*a_tolerance/2*K
    relative_coords = np.where( np.isclose(norm_f_on_grid(mesh),0,atol=threshold_f)
             & np.isclose(norm_grad_on_grid(mesh),0, atol=threshold_grad)  )
    return np.column_stack( (mesh[0][relative_coords], mesh[1][relative_coords], mesh[2][relative_coords]) ).tolist() #or use np.vstack((mesh[0][relative_coords],mesh[1][relative_coords])).T.tolist()

In [11]:
def bound_3df(init_point,B,Num_base_points,epsilon_min = 1e-6):
    epsilon = 2*B/(Num_base_points - 1)
    while epsilon > epsilon_min:
        list_dub_points = dubious_points(grid(init_point[0],init_point[1],init_point[2],B,Num_base_points), epsilon )
        print(f"epsilon={epsilon},\n Coords of dubious points={list_dub_points}")
        print(" ")
        if list_dub_points == []:
            print(epsilon)
            epsilon = 0
        else:
            B = epsilon
            epsilon = epsilon/(Num_base_points - 1)
            l = []
            [l.append(dubious_points(grid(coordinate[0],coordinate[1],coordinate[2],B,Num_base_points), epsilon) ) for coordinate in list_dub_points]
            


In [12]:
bound_3df([0,0,0],2,4)

epsilon=1.3333333333333333,
 Coords of dubious points=[[-0.6666666666666667, -2.0, -0.6666666666666667], [-0.6666666666666667, -2.0, 0.6666666666666665], [0.6666666666666665, -2.0, -0.6666666666666667], [0.6666666666666665, -2.0, 0.6666666666666665], [-2.0, -0.6666666666666667, -0.6666666666666667], [-2.0, -0.6666666666666667, 0.6666666666666665], [-0.6666666666666667, -0.6666666666666667, -2.0], [-0.6666666666666667, -0.6666666666666667, -0.6666666666666667], [-0.6666666666666667, -0.6666666666666667, 0.6666666666666665], [-0.6666666666666667, -0.6666666666666667, 2.0], [0.6666666666666665, -0.6666666666666667, -2.0], [0.6666666666666665, -0.6666666666666667, -0.6666666666666667], [0.6666666666666665, -0.6666666666666667, 0.6666666666666665], [0.6666666666666665, -0.6666666666666667, 2.0], [2.0, -0.6666666666666667, -0.6666666666666667], [2.0, -0.6666666666666667, 0.6666666666666665], [-2.0, 0.6666666666666665, -0.6666666666666667], [-2.0, 0.6666666666666665, 0.6666666666666665], [-0.